<a href="https://colab.research.google.com/github/cartmarsh/MachineLearning1/blob/main/Toxic_comment_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Outline

1. Get the tweets with tweepy
2. load the model
3. feed it the tweets

4. Make some Data Analysis

In [ ]:
!pip install tweepy==4.1.0
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 757 kB/s 
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install wordcloud
!pip install better_profanity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 3.8 MB/s 


In [ ]:
import re 

import numpy as np

import tweepy 

from tweepy import OAuthHandler 

from textblob import TextBlob 

import matplotlib.pyplot as plt

import pandas as pd

from wordcloud import WordCloud

from better_profanity import profanity

In [ ]:
consumer_key = 'fUYQzT1rSEHvHXJBVZXlFmpUo'
consumer_secret = 'ap2ibCmWZ2wannlajIXE1DWEZTiFBg4hJeUN6QoqPaSO1IUnZs' 
access_token = '840693273345822720-ESWlEVGjVX1J9r39R8By3mMIIamoMkP'
access_token_secret = 'OIpSJ8CkbCYLPXwxG9hQSDdCJCD7luIgWCkNmSnPU2wKG'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


api = tweepy.API(auth)

In [ ]:
tweepy.__version__

'4.1.0'

In [ ]:
# Input a query from the user

query = input("Please enter your topic / person of interest: ")

# In this case, we will input the query as 'Elon Musk'

# Filter the query to remove retweets

filtered = query + "-filter:retweets"

# Generate the latest tweets on the given query 

tweets = tweepy.Cursor(api.search_tweets, 
                           q=filtered,
                           lang="en").items(100)

# Create a list of the tweets, the users, and their location

list1 = [[tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]

Please enter your topic / person of interest: musk


In [ ]:
# Convert the list into a dataframe

df = pd.DataFrame(data=list1, 
                    columns=['tweets','user', "location"])
# Convert only the tweets into a list

tweet_list = df.tweets.to_list()

In [ ]:
tweet_list[:10]

['@CollinRugg Keep up the good fight, Mr. Musk',
 "Thank God for Mr. Musk. Defund the DOJ until it's made pro American.",
 "@AmoneyResists @elonmusk @Unfilteredboss1 @KimDotcom Elon Musk won't be able to sell Twitter to anyone without taki… https://t.co/YEJpoX95zx",
 'Sensible analysis of man child Musk.   Fintan O’Toole: Musk’s tantrum capitalism explodes myth of  buccaneering lib… https://t.co/o2Jd3qEwea',
 "Elon Musk's Stalker Is an Uber Eats Driver Who Says Musk Is Stalking Him https://t.co/DkkQqNo5s3 https://t.co/M8evdnBkFc",
 "Muskie can't chew with his mouth full, eh?\n#twitter #money\nI ✍️ \n\nhttps://t.co/CNHUuCox5I",
 '@transboyautism @doamnemiluieste yea but that is just twitter. it is like a breeding ground for weird lolisho artis… https://t.co/QSOvID9VaF',
 '@AGblueMetal @MUSK suspend him fr',
 'Set of 2 - Concentrated Fragrance Oil - Scent - Black Tie: Sophisticated notes of leather w/warm woods, patchouli,… https://t.co/nNHKqqk9pf',
 '@HASOsyalmedya This feels a lot like

We can improve the stopwords here by donwloading a library

In [ ]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    r = tweet.lower()
    r = profanity.censor(r)
    r = re.sub("'", "", r) # This is to avoid removing contractions in english
    r = re.sub("@[A-Za-z0-9_]+","", r)
    r = re.sub("#[A-Za-z0-9_]+","", r)
    r = re.sub(r'http\S+', '', r)
    r = re.sub('[()!?]', ' ', r)
    r = re.sub('\[.*?\]',' ', r)
    r = re.sub("[^a-z0-9]"," ", r)
    r = r.split()
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    r = [w for w in r if not w in stopwords]
    r = " ".join(word for word in r)
    return r

In [ ]:
cleaned = [clean_tweet(tw) for tw in tweet_list]
cleaned[:10]

['keep up good fight mr musk',
 'thank mr musk defund doj until its made pro american',
 'elon musk wont be able sell twitter anyone without taki',
 'sensible analysis man child musk fintan o toole musk s tantrum capitalism explodes myth buccaneering lib',
 'elon musks stalker is uber eats driver who says musk is stalking him',
 'muskie cant chew with his mouth full eh i',
 'yea but that is just twitter it is like breeding ground weird lolisho artis',
 'suspend him fr',
 'set 2 concentrated fragrance oil scent black tie sophisticated notes leather w warm woods patchouli',
 'this feels lot like christmas support crypto community tesla initiated 5 000 100']

Load the model

In [ ]:
import torch

In [ ]:
!pwd


/content


In [ ]:
!ls

sample_data


In [ ]:
model = torch.load("./checkpointRNN_toxic_comment_classifier.ckpt")

In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 10.0 MB/s 
     |████████████████████████████████| 125 kB 74.7 MB/s 
     |████████████████████████████████| 512 kB 67.8 MB/s 


In [ ]:
import pytorch_lightning as pl

Prepare the data for prediction

In [ ]:
from torchtext.data.utils import get_tokenizer

In [ ]:
tokenizer = get_tokenizer("basic_english")

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
comments_token = [tokenizer(comment) for comment in cleaned]

In [ ]:
comments_token = pd.DataFrame(comments_token)

In [ ]:
comments_token[:10]

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,keep,up,good,fight,mr,musk,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,thank,mr,musk,defund,doj,until,its,made,pro,american,...,None,None,None,None,None,None,None,None,None,None
2,elon,musk,wont,be,able,sell,twitter,anyone,without,taki,...,None,None,None,None,None,None,None,None,None,None
3,sensible,analysis,man,child,musk,fintan,o,toole,musk,s,...,buccaneering,lib,None,None,None,None,None,None,None,None
4,elon,musks,stalker,is,uber,eats,driver,who,says,musk,...,None,None,None,None,None,None,None,None,None,None
5,muskie,cant,chew,with,his,mouth,full,eh,i,None,...,None,None,None,None,None,None,None,None,None,None
6,yea,but,that,is,just,twitter,it,is,like,breeding,...,None,None,None,None,None,None,None,None,None,None
7,suspend,him,fr,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,set,2,concentrated,fragrance,oil,scent,black,tie,sophisticated,notes,...,patchouli,None,None,None,None,None,None,None,None,None
9,this,feels,lot,like,christmas,support,crypto,community,tesla,initiated,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
VOCAB_SIZE = 1500

In [ ]:
## for words that are unknown to the vocab
unk_token = ""

## to pad sentences, because we need the same length for RNNs
pad_token = ""

In [ ]:
vocab = build_vocab_from_iterator(comments_token, specials=[unk_token, pad_token],
                                  max_tokens=VOCAB_SIZE)

TypeError: ignored

In [ ]:
from torch.utils.data import DataLoader